In [22]:
import os
import re
import shutil
from multiprocessing import Pool
import requests
from astropy.io import fits
from astropy.wcs import WCS
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output
import requests
from astroquery.mast import Tesscut
from astropy.coordinates import SkyCoord
import imageio
from os import listdir
from os.path import isfile, join

image_dir = 'E:/ffi/obs3/'
script_url = "https://archive.stsci.edu/missions/tess/download_scripts/sector"
script_url += "/tesscurl_sector_{0}_ffic.sh"

def get_sector_table(ra,dec):
    coord = SkyCoord(ra,dec,unit='deg')
    sector_table = Tesscut.get_sectors(coordinates=coord)
    print(sector_table)
    
def get_url_list(sector, camera=None, chip=None):
    """Get the list of FFI URLs by fetching the MAST bulk download script"""

    # Construct the regular expression pattern
    pattern = "-s{0:04d}-".format(sector)
    if camera is None:
        pattern += "([1-4]+)-"
    else:
        pattern += "{0:d}-".format(camera)
    if chip is None:
        pattern += "([1-4]+)-"
    else:
        pattern += "{0:d}-".format(chip)
    prog = re.compile(pattern)

    # Download the bulk download script
    r = requests.get(script_url.format(sector))
    r.raise_for_status()

    # Loop over lines in the script and construct the right filenames
    urls = []
    for line in r.text.splitlines():
        if line.startswith("#"):
            continue
        if len(prog.findall(line)) == 0:
            continue
        line = line.strip()
        fn, url = line.split()[-2:]
        year = fn[4:8]
        num = fn[8:11]
        sector, chip, ccd = fn.split("-")[1:4]
        path = "tess/ffi/{0}/{1}/{2}/{3}-{4}/{5}".format(sector, year, num,
                                                         chip, ccd, fn)
        urls.append(url)

    return urls

def make_images(filenames):
    print('There are ' + str(len(filenames)) + ' images to make.')
    numb = 0
    for fits_file in filenames:
        fits_file = fits_file[69:]
        with fits.open(image_dir + fits_file, mode = "readonly") as hdulist:
            wcs_info = WCS(hdulist[1].header)
            cal_image = hdulist[1].data
            header = hdulist[1].header

        mid_time = header['TSTART']

        plt.figure(figsize = (8,8))

        plt.subplot(111, projection = wcs_info)
        plt.imshow(cal_image, vmin = np.percentile(cal_image,4),vmax = np.percentile(cal_image, 98),cmap=plt.cm.bone,origin = "lower")
        plt.xlabel('RA')
        plt.ylabel('Dec')
        plt.title("TESS Calibrated FFI for Sector 1, Camera 4, CCD 2, Timestamp %f BTJD" % mid_time)
        plt.savefig(image_dir + str(mid_time)+'.png',dpi=300)
        plt.close()
        clear_output()
        del wcs_info, cal_image, header
        print(str(numb) + " / " + str(len(filenames)) + "     " + str(100*numb/len(filenames)) + '%')
        numb = numb + 1
    clear_output()
    print("Images generated.")
        
def download(filenames):
    numb = 0
    for file in filenames:
        myfile = requests.get(file)
        open(image_dir + str(file)[69:], 'wb').write(myfile.content)
        numb = numb + 1
        clear_output()
        print(str(numb) + " / " + str(len(filenames)) + "     " + str(100*numb/len(filenames)) + '%')
        
def make_movie():
    filenames = []
    for files in listdir(image_dir):
        filenames.append(image_dir + files)
    with imageio.get_writer(image_dir + '/17-1-2.mp4', mode='I',fps=10) as writer:
        num = 0
        for filename in filenames:
            try:
                image = imageio.imread(filename)
                writer.append_data(image)
                clear_output()
                print(num)
                num = num + 1
            except:
                pass

In [23]:
get_sector_table(0,29.75)

  sectorName   sector camera ccd
-------------- ------ ------ ---
tess-s0017-1-2     17      1   2


In [24]:
filenames = get_url_list(17,camera=1,chip=1)[1:1200:5]
download(filenames)
make_images(filenames)
make_movie()

112 / 226     49.557522123893804%


OSError: Header missing END card.

In [25]:
make_movie()

112


In [12]:
pip install imageio-ffmpeg

  Using cached imageio_ffmpeg-0.4.3-py3-none-win_amd64.whl (22.6 MB)
Note: you may need to restart the kernel to use updated packages.
